In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#using pandas to read the csv file and save it in dataframe
sample_submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/sample_submission.csv')
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/test.csv')


In [ ]:
#printing the head of dataframe
sample_submission_df.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
#printing the train data head to check values
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#lets check for the maximum value in the columns
train_df.to_numpy().max()

255

In [ ]:
#check the info of the dataframe which will include total number of columns, data size and label information
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [ ]:
#lets check for the shape of our train dataset
train_df.shape

(42000, 785)

In [ ]:
#import categorical from keras.utils library
from tensorflow.keras.utils import to_categorical

#extract the labels from the data and convert it to categorical data
train_labels = np.asarray(train_df['label']);
train_labels = to_categorical(train_labels);


In [ ]:
train_labels

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
#Drop the labels from the train dataset and shape it in 28x28 size
train_data = train_df.drop('label', axis=1)
# train_data = np.asarray(train_data).reshape(train_data.shape[0], 28, 28, 1)

train_data.shape

(42000, 784)

In [ ]:
# convert from integers to floats
train_norm = train_data.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0

train_norm.shape

(42000, 784)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, y_train, X_rem, y_rem = train_test_split(train_norm,train_labels, train_size=0.8)

y_rem.shape

(8400, 10)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization

In [ ]:
model = Sequential();
model.add(Dense(784,activation='relu', input_shape=(784,)))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(Dense(10, activation='softmax'))
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, X_rem, epochs=10, batch_size=32, validation_data=(y_train, y_rem), verbose=0)

In [ ]:
history.history['loss']

[0.23673611879348755,
 0.1082736924290657,
 0.08136078715324402,
 0.056299224495887756,
 0.04634057730436325,
 0.041295912116765976,
 0.03230419009923935,
 0.027466103434562683,
 0.026432719081640244,
 0.022177398204803467]

In [ ]:
predictions = model.predict(test_df)

In [ ]:
predictions

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
y_classes = [np.argmax(y, axis=None, out=None) for y in predictions]

len(y_classes)

28000

In [ ]:
ImageId = [n for n in range(1,28001)]
len(ImageId)

28000

In [ ]:
data = {'ImageId':ImageId,'Label':y_classes}

In [ ]:
df = pd.DataFrame(data)


In [ ]:
df.to_csv(path_or_buf='/content/drive/MyDrive/Colab Notebooks/Projects/submission.csv',index=False)



---

Trying with convolutional neural network


In [ ]:
train_data = np.asarray(train_data).reshape(train_data.shape[0], 28, 28, 1)


X_train = np.asarray(X_train).reshape(X_train.shape[0], 28, 28, 1)
y_train = np.asarray(y_train).reshape(y_train.shape[0], 28, 28, 1)

In [ ]:
X_train.shape

(33600, 28, 28, 1)

In [ ]:
X_rem.shape

(33600, 10)

In [ ]:
y_train.shape

(8400, 28, 28, 1)

In [ ]:
y_rem.shape

(8400, 10)

In [ ]:
model1 = Sequential()
model1.add(Conv2D(64, (5, 5), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model1.add(BatchNormalization())
model1.add(Conv2D(128, (5, 5), activation='relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(256, (5, 5), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(512, (5, 5), activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model1.add(BatchNormalization())
model1.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 24, 24, 64)        1664      
_________________________________________________________________
batch_normalization_16 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 20, 128)       204928    
_________________________________________________________________
batch_normalization_17 (Batc (None, 20, 20, 128)       512       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 256)       819456    
_________________________________________________________________
batch_normalization_18 (Batc (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 256)        

In [ ]:
history = model1.fit(X_train, X_rem, epochs=20, batch_size=32, validation_data=(y_train, y_rem), verbose=0)

In [ ]:
test_norm = test_df.astype('float32')/255.0
test_data = np.asarray(test_norm).reshape(test_norm.shape[0], 28, 28, 1)
predictions1 = model1.predict(test_data)

In [ ]:
y_classes = [np.argmax(y, axis=None, out=None) for y in predictions1]
ImageId = [n for n in range(1,28001)]

In [ ]:
data = {'ImageId':ImageId,'Label':y_classes}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv(path_or_buf='/content/drive/MyDrive/Colab Notebooks/Projects/submission.csv',index=False)